<a href="https://colab.research.google.com/github/Pooja-Pote18/RAGProject/blob/main/RAG_cybersecurity_doc_assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers transformers PyPDF2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.0 MB/s eta 0:00:00


In [ ]:
import PyPDF2
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import numpy as np
from google.colab import files

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [ ]:
uploaded = files.upload()  # Upload your cybersecurity PDF document

pdf_path = list(uploaded.keys())[0]
print(f"Uploaded file: {pdf_path}")

def extract_text_from_pdf(pdf_path):
    reader = PyPDF2.PdfReader(open(pdf_path, "rb"))
    full_text = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            full_text += text + "\n"
    return full_text

document_text = extract_text_from_pdf(pdf_path)
print(f"Extracted text length: {len(document_text)} characters")


Saving cybersecurity.pdf to cybersecurity.pdf
Uploaded file: cybersecurity.pdf
Extracted text length: 84666 characters


In [ ]:
def split_text(text, chunk_size=1000, overlap=150):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        # Try to avoid cutting mid-sentence by backing up to last period if possible
        if end < len(text):
            last_period = chunk.rfind('.')
            if last_period != -1 and last_period > chunk_size * 0.5:
                end = start + last_period + 1
                chunk = text[start:end]
        chunks.append(chunk.strip())
        start = end - overlap
    return chunks

chunks = split_text(document_text)
print(f"Total number of chunks: {len(chunks)}")
print("Sample chunk preview:\n", chunks[0][:500])

Total number of chunks: 112
Sample chunk preview:
 CYBER SECURITY  
[R18A05 21] 
LECTURE NOTES  
 
 
B.TECH III YEAR – II SEM (R18)  
(2020 -2021) 
 
 
 
 
 
DEPA RTMENT OF CSE  
MALLA REDDY COLLEGE OF ENGINEERING & 
TECHNOLOGY  
(Autonomous Institution – UGC, Govt. of 
India)  
Recognized under 2(f) and 12 (B) of UGC ACT 1956  
(Affiliated to JNTUH, Hyderabad, Approved by AICTE - Accredited by NBA & NAAC – ‘A’ Grade - ISO 9001:2015 
Certified)  
Maisammaguda, Dhulapally (Post Via. Hakimpet), Secunderabad – 500100, 
Telangana State, India  

MAL


In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')  # CPU safer for embeddings

chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True)
print(f"Embeddings shape: {chunk_embeddings.shape}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings shape: (112, 384)


In [ ]:
# Change model to fit your GPU, here using 'EleutherAI/gpt-neo-125M'
model_name = "EleutherAI/gpt-neo-125M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def remove_repetition(text):
    lines = text.split('\n')
    seen = set()
    filtered = []
    for line in lines:
        if line.strip() not in seen:
            filtered.append(line)
            seen.add(line.strip())
        else:
            break
    return '\n'.join(filtered)

def answer_query(query, chunks, chunk_embeddings, embedder, generator, top_k=1):
    query_emb = embedder.encode([query], convert_to_numpy=True).reshape(-1)
    similarities = np.dot(chunk_embeddings, query_emb) / (
        np.linalg.norm(chunk_embeddings, axis=1) * np.linalg.norm(query_emb) + 1e-8
    )
    top_indices = similarities.argsort()[-top_k:][::-1]

    context = "\n".join([chunks[i] for i in top_indices])
    if len(context) > 1500:
        context = context[:1500]  # Truncate to max allowed length for prompt

    prompt = (
        "You are a precise and helpful assistant specializing in government cybersecurity policies. "
        "Answer the question strictly using the context below.\n\n"
        f"Context:\n{context}\n\n"
        f"Question:\n{query}\n\n"
        "Answer:"
    )

    outputs = generator(
        prompt,
        max_new_tokens=150,
        temperature=0.3,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True
    )

    answer = outputs[0]["generated_text"]
    cleaned_answer = remove_repetition(answer)
    return cleaned_answer

# Example Question
question = "Why is cyber security important?"
print(answer_query(question, chunks, chunk_embeddings, embedder, text_gen))


You are a precise and helpful assistant specializing in government cybersecurity policies. Answer the question strictly using the context below.

Context:
or 
data.  
• Whereas security related to the protection which includes systems security, network 
security and application and information security.  
Why is cyber security important?                                                                                                                                                                                                                                                                                                                                                                         
Listed below are the reasons why cyber security is so important in  what’s become a 
predominant digital world:  
• Cyber attacks can be extremely expensive for businesses to endure.  
• In addition to financial damage suffered by the business, a  data breach can also inflict 
untold reputational d